In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import imblearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import trange
import seaborn as sns
import os
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import sklearn
import time
import datetime
import random

plt.style.use("seaborn")
torch.set_default_dtype(torch.float32)

In [2]:
from comet_ml import Experiment, Optimizer

In [3]:
import pyhopper

In [4]:
from loguru import logger

logger.add("log.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

1

In [5]:
from hypertab_benchmark_utils import *

# GLOBALS

In [6]:
# DEVICE="cuda:0"
# TEST_RUN = False
# DATA = "Hill-Valley-with-noise"

In [7]:
GS_METRIC = "balanced_accuracy"

In [8]:
SEED = 42

In [9]:

os.environ["HYPERTAB_TEST_RUN"] = str(TEST_RUN)

In [10]:
TIME_BUDGET = "30m"

In [ ]:
FILE_NAME = f"{DATA}_uci3_metrics_{str(time.time())}.csv"

# Load data

In [12]:
set_seed(SEED)

In [13]:
_X, _y = get_data(DATA)

train (606, 100) 2
test (606, 100) 2


In [14]:
n_classes = get_n_classes(_X, _y)
n_features = get_n_features(_X, _y)

n_classes 2
n_features 100


In [15]:
get_each_class_counts(_X, _y)

class counts {0: 606, 1: 606}


{0: 606, 1: 606}

# Split

In [16]:
X_train, X_test, y_train, y_test = initial_split(_X, _y)

X = X_train, X_test
y = y_train, y_test

In [17]:
n_classes = get_n_classes(X_train, y_train)
n_features = get_n_features(X_train, y_train)

n_classes 2
n_features 100


In [18]:
get_each_class_counts(X_train, y_train)

class counts {0: 307, 1: 299}


{0: 307, 1: 299}

In [19]:
eval_train_max_size, train_max_size = get_eval_and_benchmark_size(X_train=X_train)

eval_max_size 606
train_max_size 454


# TRAIN MODELS

## Common hyperparams

In [20]:
_mask_sizes = [int(n_features*0.1), int(n_features*0.25), int(n_features*0.5), int(n_features*0.7), int(n_features*0.9)]
if 2 not in _mask_sizes:
    _mask_sizes.insert(0, 2)
    
print('_mask_sizes', _mask_sizes)

class CommonHyperparams:
    lr = pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1])
    batch_size = pyhopper.choice([32, 64, 128])
    ht_mask_size = pyhopper.choice(_mask_sizes)
    ht_target_size = pyhopper.choice([5, 10, 20, 50])
    ht_mask_no = pyhopper.choice([5, 10, 50, 100, 150, 200, 300])
    ht_epochs = pyhopper.choice([100, 200])
    ht_first_hidden = pyhopper.choice([32, 64, 256, 512])
    

_mask_sizes [2, 10, 25, 50, 70, 90]


# GPU

## Dropout Neural network

#### Find Hyperparams

### Dropout 1 layer

In [24]:
set_seed(SEED)

network_fn1=get_parametrized_dropout_net1(DEVICE=DEVICE, n_features=n_features, n_classes=n_classes)

In [25]:
param_grid = {
                "epochs": CommonHyperparams.ht_epochs,
                "lr": CommonHyperparams.lr,
                "batch_size": CommonHyperparams.batch_size,
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
             }

nn_fn1_best_params, nn_fn1_history = pyhopper_best_params(
    model_fn=network_fn1,
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device=DEVICE,
    time=TIME_BUDGET,
)

nn_fn1_best_params

THIS IS TEST RUN
| DEVICE: cpu
| model_fn network_fn1

pyhopper X.shape: (606, 100) y.shape: (606,) train_size: 454


  0%|          | [00:00<?]

Search is scheduled for 01:00 (m:s)
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 32, 'drop1': 0.3, 'drop2': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.3, 'batch_size': 64, 'drop1': 0.7, 'drop2': 0.7}
iter 1 of 1 X_train shape torch.Size([454, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


params {'epochs': 10, 'lr': 0.03, 'batch_size': 64, 'drop1': 0.7, 'drop2': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.03, 'batch_size': 64, 'drop1': 0.1, 'drop2': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.3, 'batch_size': 64, 'drop1': 0.7, 'drop2': 0.5}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 32, 'drop1': 0.1, 'drop2': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 32, 'drop1': 0.1, 'drop2': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 32, 'drop1': 0.1, 'drop2': 0.5}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.3, 'batch_size': 64, 'drop1': 0.1, 'drop2': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 64, 'drop1': 0.5, 'drop2': 0.1}
iter 1 of 1 X

...

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


params {'epochs': 10, 'lr': 0.03, 'batch_size': 32, 'drop1': 0.1, 'drop2': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.3, 'batch_size': 64, 'drop1': 0.1, 'drop2': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 32, 'drop1': 0.3, 'drop2': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 64, 'drop1': 0.1, 'drop2': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 64, 'drop1': 0.1, 'drop2': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.3, 'batch_size': 64, 'drop1': 0.3, 'drop2': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


params {'epochs': 10, 'lr': 3e-05, 'batch_size': 64, 'drop1': 0.3, 'drop2': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.3, 'batch_size': 32, 'drop1': 0.1, 'drop2': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.03, 'batch_size': 64, 'drop1': 0.3, 'drop2': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
================= Summary =================
Mode              : Best f : Steps : Time  
----------------  : ----   : ----  : ----  
Initial solution  : 50     : 1     : 1.46 s
Random seeding    : 53.61  : 19    : 29 s  
Local sampling    : 54.94  : 17    : 29 s  
Duplicates        : -      : 58    : -     
----------------  : ----   : ----  : ----  
Total             : 54.94  : 95    : 60 s  
Hill-Valley-with-noise_network_fn1_{'epochs': 10, 'lr': 0.003, 'batch_size': 64, 'drop1': 0.1, 'drop2': 0.1}


{'epochs': 10, 'lr': 0.003, 'batch_size': 64, 'drop1': 0.1, 'drop2': 0.1}

In [26]:
nn1_results = test_model(
    model_fn=network_fn1(**nn_fn1_best_params),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

iter 1 of 1 X_train shape torch.Size([606, 100])


In [27]:
print_mean_std_max(nn1_results, DATA)

metric balanced_accuracy
dataset_name Hill-Valley-with-noise
Hill-Valley-with-noise: 50.15 ~ nan (max: 50.15)


### Dropout 2 layers

In [28]:
set_seed(SEED)

network_fn2=get_parametrized_dropout_net2(DEVICE=DEVICE, n_features=n_features, n_classes=n_classes)

In [29]:
param_grid = {
                "epochs": CommonHyperparams.ht_epochs,
                "lr": CommonHyperparams.lr,
                "batch_size": CommonHyperparams.batch_size,
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
             }

nn_fn2_best_params, nn_fn2_history = pyhopper_best_params(
    model_fn=network_fn2,
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device=DEVICE,
    time=TIME_BUDGET,
)
nn_fn2_best_params

THIS IS TEST RUN
| DEVICE: cpu
| model_fn network_fn2

pyhopper X.shape: (606, 100) y.shape: (606,) train_size: 454


  0%|          | [00:00<?]

Search is scheduled for 01:00 (m:s)
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 32, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.003, 'batch_size': 96, 'drop1': 0.1, 'drop2': 0.5, 'drop3': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 256, 'drop1': 0.3, 'drop2': 0.7, 'drop3': 0.7}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 480, 'drop1': 0.7, 'drop2': 0.1, 'drop3': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 384, 'drop1': 0.1, 'drop2': 0.3, 'drop3': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 96, 'drop1': 0.7, 'drop2': 0.1, 'drop3': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 160, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.1

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


params {'epochs': 10, 'lr': 3e-05, 'batch_size': 64, 'drop1': 0.1, 'drop2': 0.5, 'drop3': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 480, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.5}
iter 1 of 1 X_train shape torch.Size([454, 100])


...

params {'epochs': 10, 'lr': 3e-05, 'batch_size': 224, 'drop1': 0.7, 'drop2': 0.7, 'drop3': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.03, 'batch_size': 416, 'drop1': 0.5, 'drop2': 0.5, 'drop3': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.03, 'batch_size': 288, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.03, 'batch_size': 160, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 192, 'drop1': 0.3, 'drop2': 0.1, 'drop3': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 160, 'drop1': 0.5, 'drop2': 0.3, 'drop3': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 224, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.1}
iter 1 of 1 X_train shape torch.Si

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


params {'epochs': 10, 'lr': 0.003, 'batch_size': 160, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 256, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.3, 'batch_size': 160, 'drop1': 0.3, 'drop2': 0.1, 'drop3': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.3, 'batch_size': 224, 'drop1': 0.1, 'drop2': 0.3, 'drop3': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 416, 'drop1': 0.5, 'drop2': 0.3, 'drop3': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 288, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.3, 'batch_size': 288, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.3}
iter 1 of 1 X_train shape torch.Size(

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epochs': 10,
 'lr': 0.0003,
 'batch_size': 256,
 'drop1': 0.3,
 'drop2': 0.3,
 'drop3': 0.1}

In [30]:
nn2_results = test_model(
    model_fn=network_fn2(**nn_fn2_best_params),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

iter 1 of 1 X_train shape torch.Size([606, 100])


In [31]:
print_mean_std_max(nn2_results, DATA)

metric balanced_accuracy
dataset_name Hill-Valley-with-noise
Hill-Valley-with-noise: 50.88 ~ nan (max: 50.88)


### Dropout 3 layers

In [32]:
set_seed(SEED)

network_fn3=get_parametrized_dropout_net3(DEVICE=DEVICE, n_features=n_features, n_classes=n_classes)

In [33]:
param_grid = {
                "epochs": CommonHyperparams.ht_epochs,
                "lr": CommonHyperparams.lr,
                "batch_size": CommonHyperparams.batch_size,
                "first_hidden_layer": CommonHyperparams.ht_first_hidden,
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop4": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
             }

nn_fn3_best_params, nn_fn3_history = pyhopper_best_params(
    model_fn=network_fn3,
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device=DEVICE,
    time=TIME_BUDGET,
)

nn_fn3_best_params

THIS IS TEST RUN
| DEVICE: cpu
| model_fn network_fn3

pyhopper X.shape: (606, 100) y.shape: (606,) train_size: 454


  0%|          | [00:00<?]

Search is scheduled for 01:00 (m:s)
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 32, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.3, 'drop4': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 320, 'drop1': 0.7, 'drop2': 0.7, 'drop3': 0.3, 'drop4': 0.7}
iter 1 of 1 X_train shape torch.Size([454, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


params {'epochs': 10, 'lr': 0.3, 'batch_size': 512, 'drop1': 0.5, 'drop2': 0.5, 'drop3': 0.5, 'drop4': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.3, 'batch_size': 128, 'drop1': 0.1, 'drop2': 0.1, 'drop3': 0.7, 'drop4': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 0.0003, 'batch_size': 64, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.5, 'drop4': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 128, 'drop1': 0.7, 'drop2': 0.7, 'drop3': 0.1, 'drop4': 0.1}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 320, 'drop1': 0.5, 'drop2': 0.7, 'drop3': 0.3, 'drop4': 0.7}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 512, 'drop1': 0.3, 'drop2': 0.5, 'drop3': 0.5, 'drop4': 0.3}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'lr': 3e-05, 'batch_size': 5

...

params {'epochs': 10, 'lr': 3e-05, 'batch_size': 32, 'drop1': 0.1, 'drop2': 0.3, 'drop3': 0.3, 'drop4': 0.5}
iter 1 of 1 X_train shape torch.Size([454, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


params {'epochs': 10, 'lr': 0.03, 'batch_size': 32, 'drop1': 0.1, 'drop2': 0.7, 'drop3': 0.1, 'drop4': 0.7}
iter 1 of 1 X_train shape torch.Size([454, 100])
==================== Summary ===================
Mode              : Best f : Steps : Time       
----------------  : ----   : ----  : ----       
Initial solution  : 50     : 1     : 3.45 s     
Random seeding    : 53.61  : 19    : 28 s       
Local sampling    : 50.93  : 8     : 30 s       
Duplicates        : -      : 4     : -          
----------------  : ----   : ----  : ----       
Total             : 53.61  : 32    : 01:01 (m:s)
Hill-Valley-with-noise_network_fn3_{'epochs': 10, 'lr': 3e-05, 'batch_size': 32, 'drop1': 0.1, 'drop2': 0.7, 'drop3': 0.3, 'drop4': 0.7}


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epochs': 10,
 'lr': 3e-05,
 'batch_size': 32,
 'drop1': 0.1,
 'drop2': 0.7,
 'drop3': 0.3,
 'drop4': 0.7}

In [34]:
nn3_results = test_model(
    model_fn=network_fn3(**nn_fn3_best_params),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

iter 1 of 1 X_train shape torch.Size([606, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
print_mean_std_max(nn3_results, DATA)

metric balanced_accuracy
dataset_name Hill-Valley-with-noise
Hill-Valley-with-noise: 50.00 ~ nan (max: 50.00)


### Hypernetwork

In [20]:
set_seed(SEED)

network_hp_fn=get_parametrized_hypertab_fn(DEVICE=DEVICE, n_features=n_features, n_classes=n_classes)

#### Find hyperparams

In [21]:
param_grid = {
    "epochs": CommonHyperparams.ht_epochs,
    "masks_no": CommonHyperparams.ht_mask_no,
    "mask_size": CommonHyperparams.ht_mask_size,
    "target_size": CommonHyperparams.ht_target_size,
    "first_hidden_layer": CommonHyperparams.ht_first_hidden,
    "lr": CommonHyperparams.lr,
    "batch_size": CommonHyperparams.batch_size,
}

hp_best_params, hp_history = pyhopper_best_params(
    model_fn=network_hp_fn, 
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device=DEVICE,
    time=TIME_BUDGET,
)

hp_best_params

THIS IS TEST RUN
| DEVICE: cpu
| model_fn network_hp_fn

pyhopper X.shape: (606, 100) y.shape: (606,) train_size: 454


  0%|          | [00:00<?]

Search is scheduled for 01:00 (m:s)
params {'epochs': 10, 'masks_no': 50, 'mask_size': 2, 'target_size': 5, 'first_hidden_layer': 32, 'lr': 3e-05, 'batch_size': 32}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 200, 'mask_size': 90, 'target_size': 5, 'first_hidden_layer': 32, 'lr': 0.03, 'batch_size': 384}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 100, 'mask_size': 2, 'target_size': 20, 'first_hidden_layer': 256, 'lr': 0.0003, 'batch_size': 448}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 200, 'mask_size': 2, 'target_size': 5, 'first_hidden_layer': 256, 'lr': 3e-05, 'batch_size': 192}
iter 1 of 1 X_train shape torch.Size([454, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


params {'epochs': 10, 'masks_no': 200, 'mask_size': 2, 'target_size': 5, 'first_hidden_layer': 512, 'lr': 0.03, 'batch_size': 448}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 50, 'mask_size': 50, 'target_size': 10, 'first_hidden_layer': 256, 'lr': 3e-05, 'batch_size': 320}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 200, 'mask_size': 2, 'target_size': 50, 'first_hidden_layer': 512, 'lr': 0.3, 'batch_size': 480}
iter 1 of 1 X_train shape torch.Size([454, 100])


...

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


params {'epochs': 10, 'masks_no': 150, 'mask_size': 20, 'target_size': 10, 'first_hidden_layer': 32, 'lr': 0.0003, 'batch_size': 64}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 300, 'mask_size': 10, 'target_size': 5, 'first_hidden_layer': 64, 'lr': 0.3, 'batch_size': 32}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 300, 'mask_size': 5, 'target_size': 5, 'first_hidden_layer': 64, 'lr': 0.03, 'batch_size': 64}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 100, 'mask_size': 2, 'target_size': 5, 'first_hidden_layer': 64, 'lr': 0.03, 'batch_size': 64}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 100, 'mask_size': 50, 'target_size': 5, 'first_hidden_layer': 64, 'lr': 0.03, 'batch_size': 96}
iter 1 of 1 X_train shape torch.Size([454, 100])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


params {'epochs': 10, 'masks_no': 200, 'mask_size': 20, 'target_size': 20, 'first_hidden_layer': 64, 'lr': 0.03, 'batch_size': 64}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 100, 'mask_size': 20, 'target_size': 5, 'first_hidden_layer': 64, 'lr': 3e-05, 'batch_size': 128}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 100, 'mask_size': 20, 'target_size': 10, 'first_hidden_layer': 64, 'lr': 0.3, 'batch_size': 64}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 300, 'mask_size': 90, 'target_size': 5, 'first_hidden_layer': 64, 'lr': 0.03, 'batch_size': 96}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 100, 'mask_size': 2, 'target_size': 5, 'first_hidden_layer': 512, 'lr': 0.03, 'batch_size': 64}
iter 1 of 1 X_train shape torch.Size([454, 100])
params {'epochs': 10, 'masks_no': 200, 'mask_size': 90, 'target_size': 10, 'first_hidden_layer': 512, 

{'epochs': 10,
 'masks_no': 200,
 'mask_size': 90,
 'target_size': 10,
 'first_hidden_layer': 512,
 'lr': 0.003,
 'batch_size': 160}

In [22]:
hp_best_params

{'epochs': 10,
 'masks_no': 200,
 'mask_size': 90,
 'target_size': 10,
 'first_hidden_layer': 512,
 'lr': 0.003,
 'batch_size': 160}

#### Train using the best hyperparams

In [23]:
hyper_results = test_model(
    model_fn=network_hp_fn(**hp_best_params, verbose=True),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

iter 1 of 1 X_train shape torch.Size([606, 100])


100%|██████████| 10/10 [00:02<00:00,  4.07it/s]


In [24]:
print_mean_std_max(hyper_results, DATA)

metric balanced_accuracy
dataset_name Hill-Valley-with-noise
Hill-Valley-with-noise: 50.14 ~ nan (max: 50.14)


## NODE

In [21]:
set_seed(SEED)

node_fn=get_parametrized_node_fn(X_train=X_train, n_classes=n_classes, n_features=n_features, DEVICE=DEVICE)

#### Tune hyperparams

In [21]:
# 'layer_dim': hp.quniform('layer_dim', 100, 1200, 100),
# 'num_layers': hp.quniform('num_layers', 1, 4, 1),
# 'depth': hp.quniform('depth', 2, 7, 1)
                    
param_grid = {
    'layer_dim': pyhopper.int(64, 512, power_of=2),
    'num_layers': pyhopper.int(1, 5),
    'depth': pyhopper.int(2, 7),
    'batch_size': CommonHyperparams.batch_size,
}

node_best, node_history = pyhopper_best_params(
    model_fn=node_fn,
    param_grid=param_grid,
    data=(X_train, y_train),
    train_size=train_max_size,
    DATA=DATA,
    device=DEVICE,
    time=TIME_BUDGET
)

node_best

THIS IS TEST RUN
| DEVICE: cpu
| model_fn node_fn

pyhopper X.shape: (606, 100) y.shape: (606,) train_size: 454


  0%|          | [00:00<?]

Search is scheduled for 01:00 (m:s)
params {'layer_dim': 256, 'num_layers': 3, 'depth': 4, 'batch_size': 32}
iter 1 of 1 X_train shape torch.Size([454, 100])


/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:113: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn("Data-aware initialization is performed on less than 1000 data points. This may cause instability."
/home/z1157095/anaconda3/en

==================== Summary ===================
Mode              : Best f : Steps : Time       
----------------  : ----   : ----  : ----       
Initial solution  : 50     : 1     : 18:24 (m:s)
----------------  : ----   : ----  : ----       
Total             : 50     : 1     : 18:24 (m:s)
Hill-Valley-with-noise_node_fn_{'layer_dim': 256, 'num_layers': 3, 'depth': 4, 'batch_size': 32}


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'layer_dim': 256, 'num_layers': 3, 'depth': 4, 'batch_size': 32}

#### Use best hyperparams

In [22]:
%%capture --no-stdout

node_results = test_model(
    model_fn=node_fn(**node_best),
    data=(X, y),
    train_size=eval_train_max_size,
    iters=10
)

iter 1 of 1 X_train shape torch.Size([606, 100])


In [23]:
print_mean_std_max(node_results, DATA)

metric balanced_accuracy
dataset_name Hill-Valley-with-noise
Hill-Valley-with-noise: 50.00 ~ nan (max: 50.00)


# Collect analytics

In [43]:
d = {}

In [44]:
if 'cpu' in DEVICE:
    d['Random forest'] = rf_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
    d['XGBoost'] = xgb_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
    d['Random Subspace'] = bagging_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
    d['Ensembles'] = ensemble_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

In [63]:
if 'cuda' in DEVICE:
    d['Hypernet'] = hyper_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
    d['Dropout_1'] = nn1_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
    d['Dropout_2'] = nn2_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
    d['Dropout_3'] = nn3_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
    d['Node'] = node_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])


In [49]:
all_models_df=pd.concat(d, axis=0)
all_models_df

mean       std        max
                Class                                            
Random forest   0                  56.856187  0.000000  56.856187
                1                  46.905537  0.000000  46.905537
                F1 score           49.655172  0.000000  49.655172
                Precision          52.747253  0.000000  52.747253
                Recall             46.905537  0.000000  46.905537
                Total              51.815182  0.000000  51.815182
                balanced_accuracy  51.880862  0.000000  51.880862
                roc_auc            54.137570  0.000000  54.137570
XGBoost         0                  52.842809  0.000000  52.842809
                1                  49.837134  0.000000  49.837134
                F1 score           50.915141  0.000000  50.915141
                Precision          52.040816  0.000000  52.040816
                Recall             49.837134  0.000000  49.837134
                Total              51.320132  0.000000  51.320132
                balanced_accuracy  51.339971  0.000000  51.339971
                roc_auc            51.981088  0.000000  51.981088
Random Subspace 0                  75.451505  4.490415  86.287625
                1                  35.244300  4.492018  44.299674
                F1 score           44.180028  3.978891  51.612903
                Precision          59.764888  4.870938  70.503597
                Recall             35.244300  4.492018  44.299674
                Total              55.082508  2.438674  58.745875
                balanced_accuracy  55.347902  2.438146  59.104725
                roc_auc            58.784766  3.421513  64.830652
Ensembles       0                  66.622074  0.752818  67.892977
                1                  36.742671  1.463182  39.087948
                F1 score           43.404559  1.170990  45.283019
                Precision          53.045168  0.468804  53.846154
                Recall             36.742671  1.463182  39.087948
                Total              51.485149  0.396650  52.145215
                balanced_accuracy  51.682372  0.383054  52.324251
                roc_auc            52.004401  0.316074  52.489841

In [66]:
all_models_df.to_csv(FILE_NAME)

In [67]:
exp = Experiment('CKx9rZkSCV9YDeW8zbPqhwMux', 'hypernet-uci3-tune')
# exp.log_parameters({"epochs": epochs, "mask_size": mask_size, "masks_no": masks_no, "data_size": data_size})
exp.add_tag(f"hypernet-tune3{DATA}")
exp.log_table(FILE_NAME, all_models_df)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: sklearn, torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/hypernet-uci-tune/149575beb906407eacd784e140ed9789



{'web': 'https://www.comet.com/api/asset/download?assetId=ebb6e7a67520408ab4c331339ea83125&experimentKey=149575beb906407eacd784e140ed9789',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=ebb6e7a67520408ab4c331339ea83125&experimentKey=149575beb906407eacd784e140ed9789',
 'assetId': 'ebb6e7a67520408ab4c331339ea83125'}

### Replace some data in existing

In [68]:
# tmp_df = pd.concat(d, axis=0)
# tmp_df = tmp_df.reset_index()
# tmp_df = tmp_df.rename(columns={tmp_df.columns[0]: DATA})

# tmp_df

In [69]:
# all_models_df = pd.read_csv(f"{DATA}_metrics.csv")
# all_models_df = all_models_df.rename(columns={all_models_df.columns[0]: DATA})
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'Hypernet'].index)
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'HypernetPCA'].index)

# all_models_df

In [70]:
# all_models_df = pd.concat([all_models_df, tmp_df])
# all_models_df

f"{DATA}_metrics.csv"

all_models_df = pd.read_csv(f"{DATA}_metrics.csv")

all_models_df.reset_index()

In [74]:
# all_models_df = all_models_df.drop(all_models_df.columns[0], axis=1)
# all_models_df

tmp = all_models_df

tmp = tmp.rename(columns={tmp.columns[0]: DATA})

tmp[tmp['Class'] == "balanced_accuracy"]